# 4-3 擷取 Indeed 網站資料分析師職缺的工作描述

> 補充教材

郭耀仁

## 關於 JSON 格式資料

> JavaScript Object Notation (JSON) 為將結構化資料 (structured data) 呈現為 JavaScript 物件的標準格式，常用於網站上的資料呈現、傳輸。JSON 資料格式與語言無關。即便源自 JavaScript，但目前很多程式語言都支援 JSON 格式資料的生成和解析。例如 Python 就有 The Python Standard Library、simplejson、pyson 等；R 有 rjson、jsonlite。

Source: <https://developer.mozilla.org/zh-TW/docs/Learn/JavaScript/Objects/JSON>

Python 具有標準套件 `json` 作為剖析的媒介：

- JSON 物件對應 Python 的 `dict` 類別
- 陣列 JSON(array of JSON) 對應 Python 的 `list` 類別

In [1]:
import json
import re
from random import randint
import requests
from bs4 import BeautifulSoup

In [2]:
# JSON 物件對應 Python 的 dict 類別
json_long_str = """
{
    "title": "The Shawshank Redemption",
    "releaseYear": 1994,
    "imdbRating": 9.2,
    "stars": ["Tim Robbins", "Morgan Freeman", "Bob Gunton"]
}
"""
top_rated_movie = json.loads(json_long_str)
print(top_rated_movie)
print(type(top_rated_movie))

{'title': 'The Shawshank Redemption', 'releaseYear': 1994, 'imdbRating': 9.2, 'stars': ['Tim Robbins', 'Morgan Freeman', 'Bob Gunton']}
<class 'dict'>


In [3]:
# 陣列 JSON(array of JSON) 對應 Python 的 list 類別
json_long_str = """
[
    {
        "title": "The Shawshank Redemption",
        "releaseYear": 1994,
        "imdbRating": 9.2,
        "stars": ["Tim Robbins", "Morgan Freeman", "Bob Gunton"]
    },
    {
        "title": "The Dark Knight",
        "releaseYear": 2008,
        "imdbRating": 9.0,
        "stars": ["Christian Bale", "Heath Ledger", "Aaron Eckhart"]
    }
]
"""
top_rated_movies = json.loads(json_long_str)
print(top_rated_movies)
print(type(top_rated_movies))

[{'title': 'The Shawshank Redemption', 'releaseYear': 1994, 'imdbRating': 9.2, 'stars': ['Tim Robbins', 'Morgan Freeman', 'Bob Gunton']}, {'title': 'The Dark Knight', 'releaseYear': 2008, 'imdbRating': 9.0, 'stars': ['Christian Bale', 'Heath Ledger', 'Aaron Eckhart']}]
<class 'list'>


## 關於自訂函式 `get_first_n_job_keys`

我們希望可以將 Indeed 查詢結果的前 n 筆職缺之 Job Keys 回傳，但不想因為更動 n 就重寫一次程式，在這裡就可以運用自訂函式的技巧，宣告一個名稱為 get_first_n_job_keys 函式，設計一個參數：`n` 並給予預設值 `n=100` 代表 Job Keys 筆數。

In [4]:
def get_first_n_job_keys(n=100):
    request_url = "https://www.indeed.com/jobs"
    first_n_job_keys = []
    for i in range(0, n, 10):
        query_string_parameters = {
            'q': 'data analyst',
            'jt': 'fulltime',
            'explvl': 'entry_level',
            'start': str(i) # i 是整數型別，在查詢字串中需要以內建函式 str() 轉換為字串型別
        }
        response = requests.get(request_url, params=query_string_parameters)
        soup = BeautifulSoup(response.text)
        ref_routes = [e.get('href') for e in soup.select('.title a')]
        job_keys = [re.split('/|\?|&|=', rr)[4] for rr in ref_routes]
        first_n_job_keys += job_keys
        rand_int = randint(5, 10)
        print("Scraping page {}...".format(i // 10 + 1))
        print("Sleeping for {} secs...".format(rand_int))
        time.sleep(rand_int)
    return first_n_job_keys

## 關於內建函式 `range`

Python 內建的函式 `range` 可以幫我們生成一個能指定起始、終止與間隔的等差序列。

- `start` 參數可以給定被包含的起始值（預設為 0）
- `stop` 參數可以給定**不被包含**的終止值（無預設，必須由使用者指定），終止的值不包含是 Python 很重要的慣例
- `step` 參數可以給定間隔（預設為 1）

In [5]:
help(range)

Help on class range in module builtins:

class range(object)
 |  range(stop) -> range object
 |  range(start, stop[, step]) -> range object
 |  
 |  Return an object that produces a sequence of integers from start (inclusive)
 |  to stop (exclusive) by step.  range(i, j) produces i, i+1, i+2, ..., j-1.
 |  start defaults to 0, and stop is omitted!  range(4) produces 0, 1, 2, 3.
 |  These are exactly the valid indices for a list of 4 elements.
 |  When step is given, it specifies the increment (or decrement).
 |  
 |  Methods defined here:
 |  
 |  __bool__(self, /)
 |      self != 0
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(self, key, /)
 |      Return self[key].
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __hash__(self, /)
 |

In [6]:
print(list(range(10)))       # [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
print(list(range(1, 10, 2))) # [1, 3, 5, 7, 9]
print(list(range(0, 10, 2))) # [0, 2, 4, 6, 8]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[1, 3, 5, 7, 9]
[0, 2, 4, 6, 8]


## 關於正規表達式（Regular Expression）

> 正規表達式是被用來比對字串中特徵的模式，通常被用來搜尋、替換符合某特徵模式的文字，許多程式設計語言都支援利用正規表達式進行字串操作。

Source: <https://en.wikipedia.org/wiki/Regular_expression>

Python 具有標準套件 `re` 作為運用正規表達式進行字串操作的媒介，在 `get_first_n_job_keys` 函式的前半段我們已經將職缺搜尋頁面的連結擷取出來：

In [7]:
request_url = "https://www.indeed.com/jobs"
query_string_parameters = {
    'q': 'data analyst',
    'jt': 'fulltime',
    'explvl': 'entry_level',
    'start': '0'
}
response = requests.get(request_url, params=query_string_parameters)
soup = BeautifulSoup(response.text)
ref_routes = [e.get('href') for e in soup.select('.title a')]
print(ref_routes)

['/rc/clk?jk=f13ffd0ea121b4d8&fccid=978d9fd9799d55a8&vjs=3', '/rc/clk?jk=2532431132ffbda1&fccid=30c994b6c2f39768&vjs=3', '/rc/clk?jk=c676a737179fa1c3&fccid=978d9fd9799d55a8&vjs=3', '/rc/clk?jk=3a376da0784075d1&fccid=210934531891f9be&vjs=3', '/rc/clk?jk=c04a8d5fa0e7869c&fccid=ed3db458f28ca6b3&vjs=3', '/rc/clk?jk=2871ed5f3065b89c&fccid=9f7b4a9692015a8a&vjs=3', '/rc/clk?jk=3634e45314aa3499&fccid=8ce3cf8e1e3ddd14&vjs=3', '/rc/clk?jk=45b9fc153445c7c9&fccid=2a341562d64c7cdb&vjs=3', '/rc/clk?jk=0fcfb9089f1b9810&fccid=3bed49f19827e049&vjs=3', '/rc/clk?jk=94d68f06028bea2d&fccid=2a341562d64c7cdb&vjs=3']


如果希望將這些連結中的 `jk=` 單獨留下來，我們就要利用正規表達式比對特徵：`/|\?|&|=`，也就是碰到 `/`、`?`、`&` 或 `=` 都要切割開來：

In [8]:
split_pattern = '/|\?|&|='
splited_ref_route = re.split(split_pattern, ref_routes[0]) # 以第零個連結示範
print(splited_ref_route) # Job Key 會落在索引值 4 的位置
print(splited_ref_route[4])

['', 'rc', 'clk', 'jk', 'f13ffd0ea121b4d8', 'fccid', '978d9fd9799d55a8', 'vjs', '3']
f13ffd0ea121b4d8


## 關於 `randint` 函式

隸屬於標準套件 `random` 中的 `randint` 函式可以回傳一個介於參數 `a` 與 `b` 之間的隨機整數（包含 `a` 與 `b`）。我們利用它隨機產生一個介於 5 到 10 之間的整數，讓程式在抓下一個頁面前休息一下，保持對 Indeed 網站的禮儀。

In [9]:
help(randint)

Help on method randint in module random:

randint(a, b) method of random.Random instance
    Return random integer in range [a, b], including both end points.



In [10]:
print(randint(5, 10))

9
